# Distribución de cargas simulado con algoritmo genetico con número enteros

## Pasos
- Dividir la carga entre los procesadores y asignar la cantidad restante
- Establecer la ventana, también conocida cómo la cantidad de procesos
- Asignar las cargas
- Identificar el procesador más cargado y su número de cargas

In [8]:
from algoritmo_genetico import AlgoritmoGenetico
import multiprocessing

In [9]:
class DivisionCargasAGInt(AlgoritmoGenetico):
    def __init__(self, ventana: list, num_procesadores: int, underload: int, overload: int, low_val=0, top_val=1, num_cromosomas=50, num_genes=20, num_generaciones=200, prob_cruce=0.8, prob_mutacion=0.05, porcentaje_elitismo=0.05):
        super().__init__(low_val, top_val, num_cromosomas, num_genes,
                         num_generaciones, prob_cruce, prob_mutacion, porcentaje_elitismo)
        self.ventana = ventana
        self.num_procesadores = num_procesadores
        self.underload = underload
        self.overload = overload

    def evaluar_fitness(self, cromosoma: list):
        carga_por_procesador = [0] * self.num_procesadores

        for i, proceso in enumerate(self.ventana):
            carga_por_procesador[cromosoma[i]] += proceso

        self.carga_maxima = max(carga_por_procesador)
        cargas_aceptadas = sum(1 for carga in carga_por_procesador if self.underload < carga < self.overload)
        promedio_carga = sum(carga / self.carga_maxima for carga in carga_por_procesador) / self.num_procesadores

        return (1.0 / self.carga_maxima) * promedio_carga * (cargas_aceptadas / self.num_procesadores)
    
    def best(self, poblacion: list):
        return min(poblacion, key=self.evaluar_fitness)


## Parametros

In [10]:
n_procesos = 3600
n_procesadores = multiprocessing.cpu_count()
overloaded = (n_procesos // n_procesadores) + 100
size_ventana = 54

## Implementacion

In [11]:
ventana = [n_procesos // size_ventana] * size_ventana
residuo = n_procesos % n_procesadores
if residuo > 0:
    ventana.append(residuo)

division_cargas_ag = DivisionCargasAGInt(
    ventana = ventana,
    num_procesadores = n_procesadores, 
    underload = 50, overload = overloaded,
    top_val = n_procesadores - 1, 
    num_genes = size_ventana, num_cromosomas = 100)

poblacion = division_cargas_ag.fit()
best = division_cargas_ag.best(poblacion)

print("La mejor division de cargas es:\n", best)
print("Fitness: ", division_cargas_ag.evaluar_fitness(best))

print("\nPoblacion:")
for i, individuo in enumerate(poblacion, start = 1):
    print(f"{i}: {individuo}\n Fitness : {division_cargas_ag.evaluar_fitness(individuo)}")


La mejor division de cargas es:
 [3, 1, 0, 3, 5, 9, 3, 10, 4, 9, 8, 5, 9, 8, 6, 5, 10, 2, 8, 8, 5, 2, 5, 6, 8, 8, 8, 10, 3, 5, 3, 11, 7, 3, 5, 3, 3, 8, 7, 4, 9, 7, 8, 5, 5, 6, 0, 10, 0, 6, 4, 8, 11, 8]
Fitness:  0.00042261457550713736

Poblacion:
1: [10, 11, 8, 2, 5, 5, 3, 2, 10, 1, 3, 9, 5, 6, 6, 6, 1, 2, 9, 0, 10, 9, 7, 0, 4, 7, 3, 5, 6, 7, 0, 10, 9, 3, 8, 7, 5, 8, 7, 4, 9, 11, 11, 6, 11, 1, 8, 0, 1, 4, 4, 3, 0, 2]
 Fitness : 0.002727272727272728
2: [6, 1, 8, 2, 5, 5, 3, 10, 1, 11, 4, 9, 9, 4, 1, 10, 10, 6, 10, 2, 2, 3, 9, 0, 8, 6, 8, 10, 0, 5, 9, 11, 5, 3, 5, 7, 2, 8, 7, 4, 9, 11, 11, 6, 11, 1, 8, 0, 1, 4, 4, 3, 0, 2]
 Fitness : 0.0027272727272727275
3: [10, 11, 8, 2, 5, 5, 3, 2, 10, 1, 3, 9, 5, 6, 6, 6, 1, 2, 9, 0, 10, 9, 7, 0, 4, 7, 3, 5, 6, 7, 0, 10, 9, 3, 8, 7, 2, 8, 7, 4, 9, 11, 11, 6, 11, 1, 8, 0, 1, 4, 4, 3, 0, 2]
 Fitness : 0.0027272727272727275
4: [6, 1, 8, 2, 5, 4, 3, 10, 1, 11, 4, 9, 9, 4, 1, 10, 10, 6, 10, 2, 2, 3, 9, 0, 8, 6, 8, 10, 0, 5, 9, 11, 5, 3, 5, 7, 2, 8, 7, 4, 